In this notebook, I present the implementation of a large neural network architecture for the purpose of training a Go-playing bot. The architecture consists of convolutional, dense layers, and regularization techniques designed to process the board state of a 19x19 Go game. The goal is to predict optimal moves based on the current game state by leveraging 100,000 board game positions that represent a diverse range of scenarios.

# Import Library

The baseline accuracy is .28% since, on a 19 x 19 board, the chance of making the correct play is only 1 in 361.

In [4]:
import matplotlib.pyplot as plt
from dlgo.data.parallel_processor import GoDataProcessor
from dlgo.encoders.sevenplane import SevenPlaneEncoder
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, ZeroPadding2D, Dropout, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adagrad

# NN Function

In [2]:
def large(input_shape):
    return [
        ZeroPadding2D((3, 3), input_shape=input_shape, data_format='channels_last'),
        Conv2D(64, (7, 7), padding='valid', data_format='channels_last'),
        Activation('relu'),

        ZeroPadding2D((2, 2), data_format='channels_last'),
        Conv2D(64, (5, 5), data_format='channels_last'),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2), data_format='channels_last'),
        Dropout(0.25),

        ZeroPadding2D((2, 2), data_format='channels_last'),
        Conv2D(64, (5, 5), data_format='channels_last'),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2), data_format='channels_last'),
        Dropout(0.25),

        ZeroPadding2D((2, 2), data_format='channels_last'),
        Conv2D(48, (5, 5), data_format='channels_last'),
        Activation('relu'),

        ZeroPadding2D((2, 2), data_format='channels_last'),
        Conv2D(48, (5, 5), data_format='channels_last'),
        Activation('relu'),

        ZeroPadding2D((2, 2), data_format='channels_last'),
        Conv2D(32, (5, 5), data_format='channels_last'),
        Activation('relu'),

        ZeroPadding2D((2, 2), data_format='channels_last'),
        Conv2D(32, (5, 5), data_format='channels_last'),
        Activation('relu'),

        Flatten(),
        Dense(1024),
        Activation('relu'),
        Dropout(0.5),
    ]

# Creating training and test generators

In [ ]:
go_board_rows, go_board_cols = 19, 19
num_classes = go_board_rows * go_board_cols
num_games = 100000

encoder = OnePlaneEncoder((go_board_rows, go_board_cols)) 

processor = GoDataProcessor(encoder=encoder.name())

generator = processor.load_go_data('train', num_games, use_generator=True)
test_generator = processor.load_go_data('test', num_games, use_generator=True)

# Defining a Keras model from NN

In [ ]:
input_shape = (encoder.num_planes, go_board_rows, go_board_cols)
network_layers = large(input_shape)
model = Sequential()
for layer in network_layers:
    model.add(layer)
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adagrad(), 
metrics=['accuracy'])

# Fitting and Evaluating Keras Models with Generators

In [ ]:
epochs = 10
batch_size = 128

history = model.fit(
    x=generator.generate(batch_size, num_classes),
    epochs=epochs,
    steps_per_epoch=generator.get_num_samples() // batch_size,
    validation_data=test_generator.generate(batch_size, num_classes),
    validation_steps=test_generator.get_num_samples() // batch_size,
    callbacks=[ModelCheckpoint('../checkpoints/small_model_epoch_{epoch}.h5')],
)

score = model.evaluate(
    x=test_generator.generate(batch_size, num_classes),
    steps=test_generator.get_num_samples() // batch_size,
)

print(f'Test loss: \n{score[0]}')
print(f'Test accuracy: \n{score[1]}')

# Plot Results

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('nn_large_plot.png');